In [1]:
import dask.array as da
import dask.dataframe as dd
import dask
import pandas as pd
import numpy as np
from constants import DEBUG_POSTGRESQL_PARQUET_FOLDER
from pathlib import Path
import networkx as nx
from tqdm.contrib.concurrent import process_map
from multiprocessing import cpu_count, Pool
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import defaultdict
from ddsketch import DDSketch

In [2]:
class QueryTemplateEncoder:
    """
    Why not sklearn.preprocessing.LabelEncoder()?

    - Not all labels (query templates) are known ahead of time.
    - Not that many query templates, so hopefully this isn't a bottleneck.
    """

    def __init__(self):
        self._encodings = {}
        self._inverse = {}
        self._next_label = 1

    def fit(self, labels):
        for label in labels:
            if label not in self._encodings:
                self._encodings[label] = self._next_label
                self._inverse[self._next_label] = label
                self._next_label += 1
        return self

    def transform(self, labels):
        return [self._encodings[label] for label in labels]

    def fit_transform(self, labels):
        return self.fit(labels).transform(labels)

    def inverse_transform(self, encodings):
        return [self._inverse[encoding] for encoding in encodings]


class QtMeta:
    def __init__(self):
        self._think_time_sketch = DDSketch()

    def record(self, think_time):
        self._think_time_sketch.add(think_time)

class DfMeta:
    SESSION_BEGIN = "SESSION_BEGIN"
    SESSION_END = "SESSION_END"

    def __init__(self):
        self.qtms = {}
        self.qt_enc = QueryTemplateEncoder()
        # Dummy tokens for session begin and session end.
        self.qt_enc.fit([self.SESSION_BEGIN, self.SESSION_END, pd.NA])

        # networkx dict_of_dicts format.
        self.transition_sessions = {}
        self.transition_txns = {}

    def augment(self, df):
        # Augment the dataframe while updating internal state.

        # Encode the query templates.
        print("Encoding query templates.")
        df["query_template_enc"] = self.qt_enc.fit_transform(df["query_template"])

        # Lagged time.
        df["think_time"] = (df["log_time"] - df["log_time"].shift(1)).shift(-1).dt.total_seconds()

        def record_thinks(row):
            qt_enc = row["query_template_enc"]
            think_time = row["think_time"]
            self.qtms[qt_enc] = self.qtms.get(qt_enc, QtMeta())
            self.qtms[qt_enc].record(think_time)

        print("Computing think times.")
        df.apply(record_thinks, axis=1)

        print("Updating transitions for sessions.")
        self._update_transition_dict(self.transition_sessions, self._compute_transition_dict("session_id"))
        print("Updating transitions for transactions.")
        self._update_transition_dict(self.transition_txns, self._compute_transition_dict("virtual_transaction_id"))

    def visualize(self, target):
        assert target in ["sessions", "txns"], f"Bad target: {target}"

        if target == "sessions":
            transitions = self.transition_sessions
        else:
            assert target == "txns"
            transitions = self.transition_txns

        def rewrite(s):
            l = 24
            return "\n".join(s[i:i + l] for i in range(0, len(s), l))

        G = nx.DiGraph(transitions)
        nx.relabel_nodes(G, {k: rewrite(dfm.qt_enc.inverse_transform([k])[0]) for k in G.nodes}, copy=False)
        AG = nx.drawing.nx_agraph.to_agraph(G)
        AG.layout("dot")
        AG.draw(f"{target}.pdf")

    @staticmethod
    def _update_transition_dict(current, other):
        for src in other:
            current[src] = current.get(src, {})
            for dst in other[src]:
                current[src][dst] = current[src].get(dst, {"weight": 0})
                current[src][dst]["weight"] += other[src][dst]["weight"]
                # Set the label for printing.
                current[src][dst]["label"] = current[src][dst]["weight"]

    def _compute_transition_dict(self, group_key):
        assert group_key in ["session_id", "virtual_transaction_id"], f"Unknown group key: {group_key}"

        group_fn = None
        if group_key == "session_id":
            group_fn = self._group_session
        elif group_key == "virtual_transaction_id":
            group_fn = self._group_txn
        assert group_fn is not None, "Forgot to add a case?"

        transitions = {}
        groups = df.groupby(group_key)
        chunksize = max(1, len(groups) // cpu_count())
        grouped = process_map(group_fn, groups, chunksize=chunksize, desc=f"Grouping on {group_key}.", disable=True)
        # TODO(WAN): Parallelize.
        for group_id, group_qt_encs in tqdm(grouped, desc=f"Computing transition matrix for {group_key}.",
                                            disable=True):
            for transition in zip(group_qt_encs, group_qt_encs[1:]):
                src, dst = transition
                transitions[src] = transitions.get(src, {})
                transitions[src][dst] = transitions[src].get(dst, {"weight": 0})
                transitions[src][dst]["weight"] += 1
                transitions[src][dst]["label"] = transitions[src][dst]["weight"]
        return transitions

    def _group_txn(self, item):
        group_id, df = item
        df = df.sort_values(["log_time", "session_line_num"])
        qt_encs = df["query_template_enc"].values
        return group_id, qt_encs

    def _group_session(self, item):
        group_id, df = item
        df = df.sort_values(["log_time", "session_line_num"])
        qt_encs = df["query_template_enc"].values
        qt_encs = np.concatenate([
            self.qt_enc.transform([self.SESSION_BEGIN]),
            qt_encs,
            self.qt_enc.transform([self.SESSION_END]),
        ])
        return group_id, qt_encs


dfm = DfMeta()
for pq_file in tqdm(sorted(list(Path(DEBUG_POSTGRESQL_PARQUET_FOLDER).glob("*.parquet"))),
                    desc="Reading Parquet files.",
                    disable=True):
    df = pd.read_parquet(pq_file)
    df["query_template"] = df["query_template"].replace("", np.nan)
    dropna_before = df.shape[0]
    df = df.dropna(subset=["query_template"])
    dropna_after = df.shape[0]
    print(f"Dropped {dropna_before - dropna_after} empty query templates in {pq_file}.")
    dfm.augment(df)
# dfm.visualize("sessions")
# dfm.visualize("txns")

Dropped 30 empty query templates in /tmp/meowquet/incompletes.csv.parquet.
Encoding query templates.
Computing think times.
Updating transitions for sessions.
Updating transitions for transactions.


In [25]:
# {dfm.qt_enc.inverse_transform([i])[0]: dfm.qtms[i]._think_time_sketch.get_quantile_value(0.5) for i in range(4,47)}

In [78]:
%matplotlib notebook
from neuralprophet import NeuralProphet
import math
def generate_forecast(df, target_timestamp, granularity=pd.Timedelta(hours=1), plot=False):
    """

    Parameters
    ----------
    df : pd.DataFrame
        The dataframe containing the source data to be forecast.
        Must contain a "query_template" column.

    target_timestamp : str
        The timestamp into the future that should be forecasted until.

    granularity : pd.Timedelta
        The granularity at which forecasting should be performed.
        For example, pd.Timedelta(hours=1) will forecast data at an hour granularity.

    plot : bool
        If true, will draw plots.

    Returns
    -------

    """

    assert "query_template" in df.columns, "Must contain a query_template column!"

    # Assumption: every transaction starts with a BEGIN.
    # Therefore, only the BEGIN entries need to be considered.
    # TODO(WAN): Other ways of starting transactions.
    begin_times = df[df["query_template"] == "BEGIN"].set_index("log_time").resample(granularity).size()

    # Convert the dataframe to NeuralProphet format.
    ndf = begin_times.iloc[:-1].to_frame()
    ndf = ndf.tz_localize(None).reset_index().rename(columns={"log_time": "ds", 0: "y"})

    # Get the forecast time range.
    ts_last = ndf["ds"].max()
    horizon_target = pd.to_datetime(target_timestamp)
    assert horizon_target > ts_last, "Horizon is not in the future?"

    # ndf_forecast is the forecast dataframe that needs to have its "y" columns filled in.
    # NeuralProphet's make_future_dataframe is not used because it doesn't seem to expose the freq.
    dr = pd.date_range(start=ts_last, end=horizon_target, freq=granularity, inclusive="right")
    ndf_forecast = pd.DataFrame({"ds": dr, "y": [None] * len(dr)})
    ndf_forecast = pd.concat([ndf, ndf_forecast])

    model = NeuralProphet()
    # Train NeuralProphet on the original data.
    metrics = model.fit(ndf, freq=granularity)
    # Forecast the future.
    forecast = model.predict(ndf_forecast)

    if plot:
        model.plot(forecast)
        model.plot_components(forecast)

    return forecast

forecast = generate_forecast(df, "2022-03-28 18:50:00", pd.Timedelta(seconds=10), plot=True)

INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling weekly seasonality. Run NeuralProphet with weekly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 6
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 500


  0%|          | 0/160 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 8.78E-02, min: 2.09E+01


  0%|          | 0/160 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 1.00E-01, min: 1.61E+01


  0%|          | 0/160 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 8.78E-02, min: 1.24E+01
INFO - (NP.forecaster._init_train_loader) - lr-range-test selected learning rate: 9.17E-02
Epoch[500/500]: 100%|██████████| 500/500 [00:00<00:00, 504.95it/s, SmoothL1Loss=0.00408, MAE=112, RMSE=170, RegLoss=0]    
WARNING - (py.warnings._showwarnmsg) - /home/kapi/.local/lib/python3.8/site-packages/neuralprophet/forecaster.py:395: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_end_to_append)



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [79]:
forecast

,ds,y,yhat1,residual1,trend
0,2022-03-28 18:47:50,4525,4804.629395,279.629395,4804.629395
1,2022-03-28 18:48:00,5828,5526.528320,-301.47168,5526.528320
2,2022-03-28 18:48:10,6047,6029.130371,-17.869629,6029.130371
3,2022-03-28 18:48:20,6159,6222.607422,63.607422,6222.607422
4,2022-03-28 18:48:30,6407,6414.801758,7.801758,6414.801758
5,2022-03-28 18:48:40,6321,6325.360840,4.36084,6325.360840
6,2022-03-28 18:48:50,None,6212.647949,NaN,6212.647949
7,2022-03-28 18:49:00,None,6099.935547,NaN,6099.935547
8,2022-03-28 18:49:10,None,5987.222656,NaN,5987.222656
9,2022-03-28 18:49:20,None,5874.509766,NaN,5874.509766


In [80]:
from tqdm import trange
with open("output.log", "w") as f:
    for i, row in forecast.iterrows():
        current_ts = row.ds
        num_forecasted_sessions = math.ceil(row.yhat1)

        # Generate a sample path for each session.
        for session_num in trange(num_forecasted_sessions, desc=f"Generating sessions for {current_ts}."):
            session_line_num = 1

            qt_cur = dfm.qt_enc.transform(["BEGIN"])[0]
            qt_ends = dfm.qt_enc.transform(["COMMIT", "ROLLBACK"])
            query = dfm.qt_enc.inverse_transform([qt_cur])[0]

            # Generate a sample path for the current session.
            sample_path = []
            while True:
                # Emit.
                sample_path.append((session_num, session_line_num, query))
                # Stop.
                if qt_cur in qt_ends:
                    break

                # Advance the session line number.
                session_line_num += 1

                # Pick the next query template by sampling the Markov chain.
                transitions = dfm.transition_txns[qt_cur].items()
                candidate_templates = [k for k, _ in transitions]
                probs = np.array([v['weight'] for _, v in transitions])
                probs = probs / np.sum(probs)
                qt_cur = np.random.choice(candidate_templates, p=probs)
                # Fill in the parameters.
                # TODO(WAN): Do that.
                query = dfm.qt_enc.inverse_transform([qt_cur])[0]
            # Write the sample path.
            for session_num, session_line_num, query in sample_path:
                print(session_num, session_line_num, query, file=f)
        # Stop after the first timestep.
        # break


Generating sessions for 2022-03-28 18:50:00.: 100%|██████████| 5424/5424 [00:04<00:00, 1160.28it/s]


In [9]:
begin_times = df[df["query_template"] == "BEGIN"].set_index("log_time").resample("L").size()

In [10]:
ndf = begin_times.iloc[:-1].to_frame()
ndf = ndf.tz_localize(None).reset_index().rename(columns={"log_time": "ds", 0: "y"})
print(ndf.shape)
display(ndf)

(59946, 2)


,ds,y
0,2022-03-28 18:47:51.158,1
1,2022-03-28 18:47:51.159,0
2,2022-03-28 18:47:51.160,0
3,2022-03-28 18:47:51.161,0
4,2022-03-28 18:47:51.162,0
...,...,...
59941,2022-03-28 18:48:51.099,1
59942,2022-03-28 18:48:51.100,0
59943,2022-03-28 18:48:51.101,0
59944,2022-03-28 18:48:51.102,1


In [11]:
%matplotlib notebook
from neuralprophet import NeuralProphet, set_random_seed

set_random_seed(15721)

num_points = ndf.shape[0]
print(num_points)

m = NeuralProphet(n_forecasts=1, n_lags=1)
test = m.fit(ndf, freq="S")
ndf_future = m.make_future_dataframe(ndf, periods=num_points*2, n_historic_predictions=True)

pred_df = ndf_future
print(pred_df)

nforecast = m.predict(pred_df)
print("Forecast")
fig_forecast = m.plot(nforecast)
plt.show()
print("Components")
fig_components = m.plot_components(nforecast)
plt.show()
print("Params")
fig_model = m.plot_parameters()
plt.show()

INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling weekly seasonality. Run NeuralProphet with weekly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 363


  0%|          | 0/191 [00:00<?, ?it/s]

59946


INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 1.81E-02, min: 4.23E+00


  0%|          | 0/191 [00:00<?, ?it/s]

...

<IPython.core.display.Javascript object>

Components


<IPython.core.display.Javascript object>

Params


<IPython.core.display.Javascript object>

In [ ]:
G = G_old

while True:
    deg_out_one = set([node for node, degree in G.out_degree if degree == 1])
    deg_in_one = set([node for node, degree in G.in_degree if degree == 1])
    contraction_candidates = list(deg_in_one & deg_out_one)

    for node in contraction_candidates:
        pred = list(G.predecessors(node))[0]
        succ = list(G.successors(node))[0]
        predw = G.in_edges[pred, node]["weight"]
        succw = G.out_edges[node, succ]["weight"]

        # Try to merge the node with its successor.
        if succ in deg_out_one and predw == succw:
            nx.contracted_nodes(G, node, succ, self_loops=False, copy=False)
            nx.relabel_nodes(G, {node: f"{node},{succ}"}, copy=False)
            break
    else:
        break
print(G)

In [ ]:
fig = plt.figure(figsize=(24, 36))
# pos = nx.nx_agraph.graphviz_layout(G)
pos = nx.spring_layout(G, weight=None)
nx.draw(G, pos, with_labels=True)
labels = nx.get_edge_attributes(G, "weight")
nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
plt.savefig("sessions_contracted.pdf")